# Qwen2.5-0.5B: Поиск оптимального слоя для QNLI


## Импорты и конфигурация

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import gc

# Конфигурация
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
MAX_LENGTH = 200
BATCH_SIZE = 16
PROBE_HIDDEN_DIM = 256
LEARNING_RATE = 1e-3
EPOCHS = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {DEVICE}")

Device: cuda


## Загрузка модели Qwen

In [2]:
# Загрузка модели
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
    output_hidden_states=True
)
model.eval()

NUM_LAYERS = model.config.num_hidden_layers
print(f"Модель имеет {NUM_LAYERS} слоёв")

2025-12-01 17:23:01.710740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764609781.905469     148 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764609781.965002     148 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Модель имеет 24 слоёв


# Загрузка датасета QNLI

In [3]:
# Загрузка данных
dataset = load_dataset("glue", "qnli")
train_data = dataset["train"].select(range(min(10000, len(dataset["train"]))))
val_data = dataset["validation"]

print(f"Train: {len(train_data)}, Val: {len(val_data)}")

README.md: 0.00B [00:00, ?B/s]

qnli/train-00000-of-00001.parquet:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

qnli/validation-00000-of-00001.parquet:   0%|          | 0.00/872k [00:00<?, ?B/s]

qnli/test-00000-of-00001.parquet:   0%|          | 0.00/877k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Train: 10000, Val: 5463


# Функции извлечения hidden states

In [4]:
def format_input(example):
    return f"Question: {example['question']}\nSentence: {example['sentence']}"

def extract_hidden_states(texts, layer):
    """Извлекаем hidden states из указанного слоя"""
    all_hidden = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), BATCH_SIZE), desc=f"Layer {layer}"):
            batch_texts = texts[i:i + BATCH_SIZE]
            
            inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH
            ).to(DEVICE)
            
            outputs = model(**inputs, output_hidden_states=True)
            hidden = outputs.hidden_states[layer]  # (batch, seq_len, hidden_dim)
            
            # Last token pooling
            seq_lengths = inputs["attention_mask"].sum(dim=1) - 1
            pooled = hidden[torch.arange(hidden.size(0), device=DEVICE), seq_lengths]
            
            all_hidden.append(pooled.cpu().float())
            
            del outputs
            torch.cuda.empty_cache()
    
    return torch.cat(all_hidden, dim=0)

## MLP Probe классификатор

In [5]:
class MLPProbe(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)

## Функция обучения

In [6]:
def train_and_evaluate(train_features, train_labels, val_features, val_labels, epochs=EPOCHS):
    """Обучаем probe и возвращаем лучшую accuracy"""
    hidden_dim = train_features.shape[1]
    
    train_loader = DataLoader(
        TensorDataset(train_features, train_labels), 
        batch_size=64, shuffle=True
    )
    val_loader = DataLoader(
        TensorDataset(val_features, val_labels), 
        batch_size=64
    )
    
    probe = MLPProbe(hidden_dim, PROBE_HIDDEN_DIM).to(DEVICE)
    optimizer = torch.optim.AdamW(probe.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()
    
    best_val_acc = 0
    best_probe_state = None
    
    for epoch in range(epochs):
        # Train
        probe.train()
        for features, labels in train_loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            loss = criterion(probe(features), labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Eval
        probe.eval()
        all_preds = []
        with torch.no_grad():
            for features, labels in val_loader:
                preds = probe(features.to(DEVICE)).argmax(dim=-1).cpu()
                all_preds.extend(preds.tolist())
        
        val_acc = accuracy_score(val_labels.tolist(), all_preds)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_probe_state = probe.state_dict().copy()
        
        print(f"  Epoch {epoch+1}: Val Acc = {val_acc:.4f}")
    
    # Возвращаем лучшую модель
    probe.load_state_dict(best_probe_state)
    return probe, best_val_acc

## Подготовка данных

In [7]:
# Подготовка текстов и меток
train_texts = [format_input(ex) for ex in train_data]
val_texts = [format_input(ex) for ex in val_data]
train_labels = torch.tensor([ex["label"] for ex in train_data])
val_labels = torch.tensor([ex["label"] for ex in val_data])

## Поиск лучшего слоя

In [8]:
# Поиск лучшего слоя
layers_to_test = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
print(f"Тестируем слои: {layers_to_test}")

results = {}

for layer in layers_to_test:
    print(f"\n{'='*40}")
    print(f"Слой {layer}")
    print('='*40)
    
    train_features = extract_hidden_states(train_texts, layer)
    val_features = extract_hidden_states(val_texts, layer)
    
    _, best_acc = train_and_evaluate(train_features, train_labels, val_features, val_labels)
    results[layer] = best_acc
    
    del train_features, val_features
    gc.collect()
    torch.cuda.empty_cache()

# Результаты поиска
print("\n" + "="*40)
print("РЕЗУЛЬТАТЫ ПОИСКА ЛУЧШЕГО СЛОЯ")
print("="*40)
for layer, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"Layer {layer:2d}: {acc:.4f}")

best_layer = max(results, key=results.get)
print(f"\nЛучший слой: {best_layer} (accuracy: {results[best_layer]:.4f})")

Тестируем слои: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

Слой 12


Layer 12: 100%|██████████| 342/342 [00:37<00:00,  9.14it/s]


  Epoch 1: Val Acc = 0.7721
  Epoch 2: Val Acc = 0.7668
  Epoch 3: Val Acc = 0.7778
  Epoch 4: Val Acc = 0.7939
  Epoch 5: Val Acc = 0.7651
  Epoch 6: Val Acc = 0.7785
  Epoch 7: Val Acc = 0.7924
  Epoch 8: Val Acc = 0.7974
  Epoch 9: Val Acc = 0.7963
  Epoch 10: Val Acc = 0.7915

Слой 13


Layer 13: 100%|██████████| 342/342 [00:37<00:00,  9.17it/s]


  Epoch 1: Val Acc = 0.8345
  Epoch 2: Val Acc = 0.8504
  Epoch 3: Val Acc = 0.8481
  Epoch 4: Val Acc = 0.8530
  Epoch 5: Val Acc = 0.8526
  Epoch 6: Val Acc = 0.8514
  Epoch 7: Val Acc = 0.8433
  Epoch 8: Val Acc = 0.8466
  Epoch 9: Val Acc = 0.8470
  Epoch 10: Val Acc = 0.8418

Слой 14


...

  Epoch 1: Val Acc = 0.8120
  Epoch 2: Val Acc = 0.8221
  Epoch 3: Val Acc = 0.8080
  Epoch 4: Val Acc = 0.8305
  Epoch 5: Val Acc = 0.7600
  Epoch 6: Val Acc = 0.8338
  Epoch 7: Val Acc = 0.8186
  Epoch 8: Val Acc = 0.8102
  Epoch 9: Val Acc = 0.8294
  Epoch 10: Val Acc = 0.8307

Слой 21


Layer 21: 100%|██████████| 342/342 [00:37<00:00,  9.13it/s]


  Epoch 1: Val Acc = 0.6738
  Epoch 2: Val Acc = 0.8078
  Epoch 3: Val Acc = 0.8107
  Epoch 4: Val Acc = 0.8083
  Epoch 5: Val Acc = 0.8113
  Epoch 6: Val Acc = 0.8188
  Epoch 7: Val Acc = 0.7937
  Epoch 8: Val Acc = 0.8041
  Epoch 9: Val Acc = 0.8105
  Epoch 10: Val Acc = 0.8036

Слой 22


Layer 22: 100%|██████████| 342/342 [00:37<00:00,  9.17it/s]


  Epoch 1: Val Acc = 0.7706
  Epoch 2: Val Acc = 0.7895
  Epoch 3: Val Acc = 0.8029
  Epoch 4: Val Acc = 0.8054
  Epoch 5: Val Acc = 0.7961
  Epoch 6: Val Acc = 0.8045
  Epoch 7: Val Acc = 0.8030
  Epoch 8: Val Acc = 0.7844
  Epoch 9: Val Acc = 0.8065
  Epoch 10: Val Acc = 0.7968

РЕЗУЛЬТАТЫ ПОИСКА ЛУЧШЕГО СЛОЯ
Layer 14: 0.8532
Layer 13: 0.8530
Layer 15: 0.8490
Layer 16: 0.8486
Layer 17: 0.8455
Layer 18: 0.8431
Layer 19: 0.8402
Layer 20: 0.8338
Layer 21: 0.8188
Layer 22: 0.8065
Layer 12: 0.7974

Лучший слой: 14 (accuracy: 0.8532)


## Финальная модель на лучшем слое

In [9]:
# Финальное обучение на лучшем слое
print(f"\n{'='*40}")
print(f"ФИНАЛЬНАЯ МОДЕЛЬ НА СЛОЕ {best_layer}")
print('='*40)

train_features = extract_hidden_states(train_texts, best_layer)
val_features = extract_hidden_states(val_texts, best_layer)

best_probe, _ = train_and_evaluate(train_features, train_labels, val_features, val_labels)


ФИНАЛЬНАЯ МОДЕЛЬ НА СЛОЕ 14


Layer 14: 100%|██████████| 342/342 [00:37<00:00,  9.15it/s]


  Epoch 1: Val Acc = 0.8060
  Epoch 2: Val Acc = 0.8492
  Epoch 3: Val Acc = 0.8528
  Epoch 4: Val Acc = 0.8539
  Epoch 5: Val Acc = 0.8517
  Epoch 6: Val Acc = 0.8455
  Epoch 7: Val Acc = 0.8514
  Epoch 8: Val Acc = 0.8466
  Epoch 9: Val Acc = 0.8494
  Epoch 10: Val Acc = 0.8393


## Метрики на обучающей выборке

In [10]:
# Метрики на TRAIN
best_probe.eval()
train_loader = DataLoader(TensorDataset(train_features, train_labels), batch_size=64)

train_preds = []
with torch.no_grad():
    for features, _ in train_loader:
        preds = best_probe(features.to(DEVICE)).argmax(dim=-1).cpu()
        train_preds.extend(preds.tolist())

print("\n" + "="*50)
print("МЕТРИКИ НА TRAIN")
print("="*50)
print(f"Accuracy: {accuracy_score(train_labels.tolist(), train_preds):.4f}")
print("\nClassification Report:")
print(classification_report(train_labels.tolist(), train_preds, target_names=["entailment", "not_entailment"]))


МЕТРИКИ НА TRAIN
Accuracy: 0.8514

Classification Report:
                precision    recall  f1-score   support

    entailment       0.92      0.77      0.84      5034
not_entailment       0.80      0.93      0.86      4966

      accuracy                           0.85     10000
     macro avg       0.86      0.85      0.85     10000
  weighted avg       0.86      0.85      0.85     10000



## Метрики на валидационной выборке

In [11]:
# Метрики на VALIDATION
val_loader = DataLoader(TensorDataset(val_features, val_labels), batch_size=64)

val_preds = []
with torch.no_grad():
    for features, _ in val_loader:
        preds = best_probe(features.to(DEVICE)).argmax(dim=-1).cpu()
        val_preds.extend(preds.tolist())

print("\n" + "="*50)
print("МЕТРИКИ НА VALIDATION")
print("="*50)
print(f"Accuracy: {accuracy_score(val_labels.tolist(), val_preds):.4f}")
print("\nClassification Report:")
print(classification_report(val_labels.tolist(), val_preds, target_names=["entailment", "not_entailment"]))
print("\nConfusion Matrix:")
print(confusion_matrix(val_labels.tolist(), val_preds))


МЕТРИКИ НА VALIDATION
Accuracy: 0.8393

Classification Report:
                precision    recall  f1-score   support

    entailment       0.89      0.77      0.83      2702
not_entailment       0.80      0.90      0.85      2761

      accuracy                           0.84      5463
     macro avg       0.84      0.84      0.84      5463
  weighted avg       0.84      0.84      0.84      5463


Confusion Matrix:
[[2092  610]
 [ 268 2493]]
